In [5]:
url = "https://www.geopython.xyz/geodata/srtm/"

urllist = [url + "srtm_53_06.zip",
    url + "srtm_53_07.zip",
    url + "srtm_54_06.zip",
    url + "srtm_54_07.zip"]


True

In [8]:
import geoutils
import os

if os.path.exists("uebung"):
    os.rmdir("uebung")


if not os.path.exists("uebung"):
    os.mkdir("uebung")

for url in urllist:
    geoutils.download(url, "uebung")

PermissionError: [Errno 13] Permission denied: 'uebung'

In [17]:
import os
import geoutils
for url in urllist:
    pfad = "uebung/" + os.path.basename(url)
    print(pfad)

Uebung1/srtm_53_06.zip
Uebung1/srtm_53_07.zip
Uebung1/srtm_54_06.zip
Uebung1/srtm_54_07.zip
